In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import os
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

import math
from copy import deepcopy

import pandas as pd

In [2]:
class DrowsinessData(Dataset):
    def __init__(self,data, trn_val_tst, transform=None):
        split_idx1 = int(data.shape[0]*0.6)
        split_idx2 = int(data.shape[0]*0.8)
        if trn_val_tst == 0:
            #trainloader
            self.X = data.iloc[0:split_idx1, 0:-3].to_numpy()
            self.labels = data.iloc[0:split_idx1, -1].to_numpy()
        elif trn_val_tst == 1:
            #valloader
            self.X = data.iloc[split_idx1:split_idx2, 0:-3].to_numpy()
            self.labels = data.iloc[split_idx1:split_idx2, -1].to_numpy()
        else:
            #testloader
            self.X = data.iloc[split_idx1:split_idx2, 0:-3].to_numpy()
            self.labels = data.iloc[split_idx1:split_idx2, -1].to_numpy()
            
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
   
        sample = self.X[idx,:]
        labels = self.labels[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, labels

In [5]:
data = pd.read_csv("features_and_labels_30s.csv")
data = data.sample(frac=1).reset_index(drop=True)

train_set = DrowsinessData(data, trn_val_tst=0, transform=transforms.ToTensor()) 
val_set = DrowsinessData(data, trn_val_tst=1, transform=transforms.ToTensor())
test_set = DrowsinessData(data, trn_val_tst=2, transform=transforms.ToTensor())

batch_size = 100 
trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

validloader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

testloader =  DataLoader(test_set, batch_size=batch_size, shuffle=False)

num_features = train_set.X.shape[1]

In [6]:
#This function should perform a single evaluation epoch, it WILL NOT be used to train our model
def evaluate(model, device, loader, loss_fun):
    #initialise counter
    epoch_acc = 0
    count = 0
    total_loss = 0
    
    #Set network in evaluation mode
    #Layers like Dropout will be disabled
    #Layers like Batchnorm will stop calculating running mean and standard deviation
    #and use current stored values
    #(More on these layer types soon!)
    model.eval()
    
    with torch.no_grad():
        for i, (x, y) in enumerate(loader):
            #Forward pass of image through network
            fx = model(x.to(device))
            y = y.type(torch.LongTensor)
            
            #log the cumulative sum of the acc
            epoch_acc += (fx.argmax(1) == y.to(device)).sum().item()
            
            #calculate the loss
            loss = loss_fun(fx, y.to(device))
            total_loss += loss.item()
            count += 1
    
    # Calculating the average loss per epoch
    loss_per_epoch = total_loss/count
    #return the accuracy from the epoch     
    return epoch_acc*100 / len(loader.dataset), loss_per_epoch

In [7]:
#This function should perform a single training epoch using our training data
def train(model, device, loader, optimizer, loss_fun, loss_logger):
    
    #Set Network in train mode
    model.train()
    
    count = 0
    total_loss = 0
    
    for i, (x, y) in enumerate(loader):
        #Perform a single epoch of training on the input dataloader, logging the loss at every step 
        #Forward pass of image through network and get output
        fx = model(x.to(device))
        y = y.type(torch.LongTensor)

        
        #Calculate loss using loss function
        loss = loss_fun(fx, y.to(device))
        
        total_loss += loss.item()
        count += 1
        
        #Zero Gradents
        optimizer.zero_grad()
        #Backpropagate Gradents
        loss.backward()
        #Do a single optimization step
        optimizer.step()

    #log the loss PER EPOCH for plotting
    loss_logger.append(total_loss/count)
   
    #return the logger array       
    return loss_logger

In [15]:
class DrowsyNet(nn.Module):
    def __init__(self, channels_in):
        # calling the init function of the parent nn.Module
        super(DrowsyNet, self).__init__()
        
        # defining the fully connected layers
        print(channels_in)
        self.fc1 = nn.Linear(int(channels_in), int(channels_in//1.5))
        self.fc2 = nn.Linear(int(channels_in//1.5), int(channels_in//3))
        self.fc3 = nn.Linear(int(channels_in//3), 1)
        
    def forward(self, x):
        # Passing it through fc layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
# Creating the model
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')
model = DrowsyNet(num_features).to(device)
print(model)

pytorch_total_params = sum([p.numel() for p in model.parameters()])
pytorch_total_trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])
print(pytorch_total_params, pytorch_total_trainable_params)

learning_rate =1e-4
start_epoch = 0
num_epochs = 100

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
loss_fun = nn.CrossEntropyLoss()

20
DrowsyNet(
  (fc1): Linear(in_features=20, out_features=13, bias=True)
  (fc2): Linear(in_features=13, out_features=6, bias=True)
  (fc3): Linear(in_features=6, out_features=1, bias=True)
)
364 364


In [17]:
training_loss_logger = []
validation_loss_logger = []

training_acc_logger = []
validation_acc_logger = []
testing_acc_logger = []

highest_val_acc = -math.inf


#This cell implements our training loop
for epoch in range(start_epoch, num_epochs):
    #call the training function and pass training dataloader etc
    training_loss_logger = train(model, device, trainloader, optimizer, loss_fun, training_loss_logger)
    
    #call the evaluate function and pass the dataloader for both ailidation and training
    train_acc, _ = evaluate(model, device, trainloader, loss_fun)
    training_acc_logger.append(train_acc)
    
    valid_acc, valid_loss = evaluate(model, device, validloader, loss_fun)
    validation_loss_logger.append(valid_loss)
    validation_acc_logger.append(valid_acc)

    test_acc, _ = evaluate(model, device, testloader, loss_fun)
    testing_acc_logger.append(test_acc)

    # Copying the model with the highest validation accuracy
    if valid_acc > highest_val_acc:
        highest_val_acc_epoch = epoch
        highest_val_acc = valid_acc
        highest_val_acc_model = deepcopy(model)
    
    clear_output(True)
    print(f'| Epoch: {epoch+1:02} | Train Acc: {train_acc:05.2f}% | Val. Acc: {valid_acc:05.2f}% | Test Acc: {test_acc:05.2f}%')
print("Training Complete")

InvalidIndexError: (1051, slice(None, None, None))